# Pipeline to train and test networks

Paths Declaration

In [5]:
RETOUCH_path = "D:\RETOUCH"
images_path = "D:\D"

Library Imports

In [6]:
from os.path import isfile, exists

Create Train-Test Split for the Fluid Segmentation Task

In [7]:
from init.foldsSplit import k_fold_split_segmentation

k = 5

if not (isfile(path="splits/segmentation_train_splits.csv") or isfile(path="splits/segmentation_test_splits.csv")):
    k_fold_split_segmentation(k=k, folders_path=RETOUCH_path)
else:
    print("Split already exists. To create a new one please delete the old files.")


Split already exists. To create a new one please delete the old files.


OCT Volumes Reading and Saving for Segmentation Task

In [8]:
from init.readOCT import save_segmentation_oct_as_tiff

if not (exists(images_path + "\\OCT_images\\segmentation\\")):
    save_segmentation_oct_as_tiff(oct_folder=RETOUCH_path, save_folder=images_path)
else:
    print("Images have already been extracted. To extract them again, please delete the folder with the images.")

Images have already been extracted. To extract them again, please delete the folder with the images.


Create Train-Test Split for the Intermediate Slice Generation Task

In [ ]:
from init.foldsSplit import k_fold_split_generation

k = 5

if not (isfile(path="splits/generation_train_splits.csv") or isfile(path="splits/generation_test_splits.csv")):
    k_fold_split_generation(k=k, folders_path=RETOUCH_path)
else:
    print("Split already exists. To create a new one please delete the old files.")

OCT Volumes Reading and Saving for Generation Task

Note: To make the generation task independent from the segmentation task, the images used for segmentation are being saved again in a different folder. For memory concerns, please adjust the code to reuse those saved previously.

In [9]:
from init.readOCT import save_generation_oct_as_tiff

if not (exists(images_path + "\\OCT_images\\generation\\")):
    save_generation_oct_as_tiff(oct_folder=RETOUCH_path, save_folder=images_path)
else:
    print("Images have already been extracted. To extract them again, please delete the folder with the images.")

Images have already been extracted. To extract them again, please delete the folder with the images.
